In [1]:
%config IPCompleter.greedy=True

In [2]:
# !pip3 install xlrd
import pandas as pd
import numpy as np
from pathlib import Path
import os
from threading import Thread
from queue import Queue
import multiprocessing
# !pip3 install khayyam
from khayyam import JalaliDate, JalaliDatetime

In [3]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"
HEADER = ["symbol", "name", "amount", "volume", "value", "lastday", "openning", "last-value",
 "last-change", "last-percent", "ending-value", "ending-change", "ending-percent",
 "min", "max",]
HEADER_extra = HEADER + ["year", "month", "day", "date"]

In [4]:
!ls $xcelLocation | grep ".xlsx" > xlFiles
tmp = !cat xlFiles
names = [name[:-5] for name in tmp]

In [5]:
def cleaner():
    for name in names:
        if os.path.getsize(xcelLocation + name + '.xlsx') < 10000:
            os.remove(xcelLocation + name + '.xlsx')
cleaner()

In [6]:
def convert(xcelLocation, xlFileName, returnAllMode=False):
    xl = None
    try:
        if (not Path(xcelLocation + xlFileName + '.csv').is_file()) or returnAllMode:
            xl = pd.read_excel(xcelLocation + xlFileName + ".xlsx",
                               header=[0], skiprows=[0,1])
            xl.columns = HEADER
            xl.to_csv(xcelLocation + xlFileName + '.csv', encoding='utf-8', index=False, header=HEADER)
    except:
        xl = str(xlFileName)
    finally:
        return xl

In [7]:
def convertThread(threadname, q, qDFs, qErrors):
    while not q.empty():
        fileNames = q.get()
        q.task_done()
        for name in fileNames:
            tmp = convert(xcelLocation=xcelLocation, xlFileName=name, returnAllMode=True)
            if isinstance(tmp, str):
                qErrors.put(tmp)
            else:
                qDFs.put((tmp.copy(), name))
    print(str(threadname) + " done")


In [8]:
allDFs = []
allDFNames = []
batchSize = 10
i = 0
numThread = 16
workers = []

pool = multiprocessing.Pool(processes=numThread)
m = multiprocessing.Manager()
queue = m.Queue()
qDFs = m.Queue()
qErrors = m.Queue()

while i*batchSize < len(names):
    if (i+1)*batchSize < len(names):
        queue.put(names[i*batchSize:(i+1)*batchSize])
    else:
        queue.put(names[i*10:])
    i+=1
print(len(names))
print(queue.qsize())

for i in range(numThread):
#     workers.append(Thread(target=readThread, args=("Thread-" + str(i), queue, qsum, qcount)))
#     workers.append(pool.apply_async(readThread, ("Thread-" + str(i), queue, qsum, qcount,)))
    workers.append(multiprocessing.Process(target=convertThread, args=("Thread-" + str(i),
                                                                    queue, qDFs, qErrors)))
    workers[i].start()

for i in range(numThread):
    workers[i].join()

while not qDFs.empty():
    dftmp, nametmp = qDFs.get()
    allDFs.append(dftmp)
    allDFNames.append(nametmp)

4422
443
Thread-9 done
Thread-14 done
Thread-10 done
Thread-15 done
Thread-2 done
Thread-1 done
Thread-4 done
Thread-8 done
Thread-3 done
Thread-0 done
Thread-7 done
Thread-12 done
Thread-13 done
Thread-11 done
Thread-5 done
Thread-6 done


In [9]:
print(len(allDFs))
def makeMasterTable(chunkSize):
    for index, df in enumerate(allDFs):
        year, month, day = allDFNames[index].split("-")
        date = JalaliDate(year, month, day).todate()
        yearlist = np.full(len(df), year).tolist()
        monthlist = np.full(len(df), month).tolist()
        daylist = np.full(len(df), day).tolist()
        datelist = np.full(len(df), date).tolist()
        df["year"] = yearlist
        df["month"] = monthlist
        df["day"] = daylist
        df["date"] = datelist
    xl = pd.concat(allDFs, keys=allDFNames, ignore_index=True)
    xl.columns = HEADER_extra
    xl = xl.astype({"year": int, "month": int, "day": int})
    xl['date'] = pd.to_datetime(xl['date'])
    print(xl.dtypes)
#     xl.sort_values(by=['year', 'month', 'day'], inplace=True)
    xl.sort_values(by=['date'], inplace=True)
    xl.reset_index(drop=True, inplace=True)
    i = 0
    while i*chunkSize < len(xl):
        if (i+1)*chunkSize < len(xl):
            df_i = xl.iloc[i*chunkSize:(i+1)*chunkSize]
        else:
            df_i = xl.iloc[i*chunkSize:]
        df_i.to_csv('{xcelLocation}master{i}.csv'.format(i=i, xcelLocation=xcelLocation),
                    header=HEADER_extra, encoding='utf-8', index=False)
        i += 1
    return xl
allDF = makeMasterTable(1000000)

4236
symbol                    object
name                      object
amount                     int64
volume                     int64
value                      int64
lastday                  float64
openning                 float64
last-value               float64
last-change              float64
last-percent             float64
ending-value             float64
ending-change            float64
ending-percent           float64
min                      float64
max                      float64
year                       int64
month                      int64
day                        int64
date              datetime64[ns]
dtype: object


In [10]:
allDF.loc[allDF["symbol"]=="خپارس"]

,symbol,name,amount,volume,value,lastday,openning,last-value,last-change,last-percent,ending-value,ending-change,ending-percent,min,max,year,month,day,date
46797,خپارس,پارس‌ خودرو,318,561712,3608122553,6000.0,6000.0,6621.0,621.0,10.35,6621.0,621.0,10.35,4000.0,10000.0,1382,2,17,2003-05-07
46971,خپارس,پارس‌ خودرو,94,124799,910907901,6952.0,7299.0,7299.0,347.0,4.99,7299.0,347.0,4.99,7299.0,7299.0,1382,2,21,2003-05-11
47165,خپارس,پارس‌ خودرو,139,179535,1375776705,7299.0,7663.0,7663.0,364.0,4.99,7663.0,364.0,4.99,7663.0,7663.0,1382,2,22,2003-05-12
47208,خپارس,پارس‌ خودرو,155,221434,1781657964,7663.0,8046.0,8046.0,383.0,5.00,8046.0,383.0,5.00,8046.0,8046.0,1382,2,23,2003-05-13
47497,خپارس,پارس‌ خودرو,104,140655,1188253440,8046.0,8448.0,8448.0,402.0,5.00,8448.0,402.0,5.00,8448.0,8448.0,1382,2,24,2003-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263674,خپارس,پارس‌ خودرو,3229,59584907,75632635055,1282.0,1268.0,1265.0,-17.0,-1.33,1269.0,-13.0,-1.01,1250.0,1297.0,1398,6,12,2019-09-03
1264384,خپارس,پارس‌ خودرو,3915,63275708,79645304743,1269.0,1269.0,1260.0,-9.0,-0.71,1259.0,-10.0,-0.79,1230.0,1284.0,1398,6,13,2019-09-04
1265129,خپارس,پارس‌ خودرو,4339,68269252,86762166478,1259.0,1265.0,1285.0,26.0,2.07,1271.0,12.0,0.95,1252.0,1289.0,1398,6,16,2019-09-07
1265940,خپارس,پارس‌ خودرو,7032,123441478,162094714785,1271.0,1282.0,1322.0,51.0,4.01,1313.0,42.0,3.30,1261.0,1334.0,1398,6,17,2019-09-08
